In [ ]:
- reddit, get liked posts
- twitter, get liked tweets
- linkedin, get liked posts


- consolidate into single format

In [ ]:
body.properties.text.files should be defined, instead was `undefined`.
body.properties.text.status should be defined, instead was `undefined`.
body.properties.text.id should be defined, instead was `undefined`.
body.properties.text.name should be defined, instead was `undefined`.
body.properties.text.start should be defined, instead was `undefined`.

In [ ]:
import requests

# Set the Github repository URL
repo_url = "https://github.com/thomaslu2000/Incremental-Parsing-Representations"


In [ ]:
from ingest.github import get_saved_repos, parse_repo

repos = get_saved_repos()


In [ ]:
repos
